<a href="https://colab.research.google.com/github/vutl/Legal-Document-Retrieval/blob/main/LDR_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
vutlol_legal_document_path = kagglehub.dataset_download('vutlol/legal-document')

print('Data source import complete.')


In [ ]:
# Import các thư viện cần thiết
!pip --upgrade install rank_bm25 transformers torch pandas sentence-transformers tqdm

In [ ]:
# Import các thư viện
import pandas as pd
from rank_bm25 import BM25Okapi
import numpy as np
from sentence_transformers import InputExample, losses, SentenceTransformer
from torch.utils.data import DataLoader
import torch.nn.functional as F
import os
from tqdm import tqdm
import re
import multiprocessing

In [ ]:
# Đường dẫn đến dữ liệu
train_data_path = "/kaggle/input/legal-document/train.csv"
corpus_path = "/kaggle/input/legal-document/corpus.csv"
test_data_path = "/kaggle/input/legal-document/public_test.csv"

In [ ]:
# Hàm tải dữ liệu Corpus
def load_corpus(corpus_path):
    corpus_df = pd.read_csv(corpus_path)
    corpus_texts = corpus_df['text'].tolist()
    corpus_ids = corpus_df['cid'].astype(str).tolist()
    return corpus_texts, corpus_ids

# Hàm tải dữ liệu huấn luyện
def load_training_data(train_data):
    training_examples = []
    for index, row in train_data.iterrows():
        question = row['question']
        related_contexts = eval(row['context'])  # Dạng list chứa các đoạn văn bản liên quan
        for context in related_contexts:
            training_examples.append(InputExample(texts=[question, context]))
    return training_examples

In [ ]:
# Hàm fine-tune mô hình Halong Embedding
def finetune_halong_embedding(train_data):
    checkpoint_path = './checkpoint'  # Đường dẫn lưu mô hình

    training_examples = load_training_data(train_data)
    train_dataloader = DataLoader(training_examples, shuffle=True, batch_size=8)
    model = SentenceTransformer("hiieu/halong_embedding")

    # Sử dụng MultipleNegativesRankingLoss cho việc huấn luyện
    train_loss = losses.MultipleNegativesRankingLoss(model)

    # Cài đặt thông số huấn luyện
    num_epochs = 1
    warmup_steps = int(0.1 * len(train_dataloader) * num_epochs)

    model.fit(train_objectives=[(train_dataloader, train_loss)],
              epochs=num_epochs,
              warmup_steps=warmup_steps,
              output_path=checkpoint_path)

    # Kiểm tra nếu checkpoint đã được lưu thành công
    if os.path.exists(checkpoint_path):
        print("Checkpoint đã được lưu thành công tại:", checkpoint_path)
    else:
        print("Lỗi: Không thể lưu checkpoint tại:", checkpoint_path)

    return model

In [ ]:
# Hàm BM25 truy xuất top_k tài liệu
def bm25_retrieve(query, bm25_model, corpus_ids, top_k=10):
    tokenized_query = query.split()
    scores = bm25_model.get_scores(tokenized_query)
    top_k_indices = np.argsort(scores)[::-1][:top_k]
    top_k_docs = [(corpus_ids[idx], scores[idx]) for idx in top_k_indices]
    return top_k_docs

In [ ]:
# Hàm tính toán MRR@10
def calculate_mrr(retrieved_results, relevant_cids_list):
    reciprocal_ranks = []
    for query_retrieved_docs, relevant_cids in zip(retrieved_results, relevant_cids_list):
        found = False
        for rank, (doc_id, _) in enumerate(query_retrieved_docs, start=1):
            if doc_id in relevant_cids:
                reciprocal_ranks.append(1 / rank)
                found = True
                break
        if not found:
            reciprocal_ranks.append(0)
    return np.mean(reciprocal_ranks)

In [ ]:
# Hàm xếp hạng lại với mô hình đã fine-tune
def rerank_with_finetuned_embedding(query, doc_ids, corpus_texts, corpus_ids, model):
    # Lấy văn bản của các tài liệu
    documents = [corpus_texts[corpus_ids.index(doc_id)] for doc_id in doc_ids]

    # Mã hóa câu hỏi và các văn bản
    query_embedding = model.encode([query], convert_to_tensor=True)
    doc_embeddings = model.encode(documents, convert_to_tensor=True)

    # Tính độ tương đồng cosine
    similarities = F.cosine_similarity(query_embedding, doc_embeddings).cpu().numpy()

    # Xếp hạng các tài liệu
    sorted_indices = np.argsort(similarities)[::-1]
    ranked_results = [(doc_ids[idx], similarities[idx]) for idx in sorted_indices]
    return ranked_results

In [ ]:
# Bỏ wandb đi phiền vl
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Hàm khởi tạo cho multiprocessing
def init_bm25(tokenized_corpus_, corpus_ids_):
    global bm25_model, corpus_ids
    bm25_model = BM25Okapi(tokenized_corpus_)
    corpus_ids = corpus_ids_

In [ ]:
# Hàm để trích xuất các số từ chuỗi trong cột 'cid'
def parse_cid_list(cid_str):
    # Sử dụng regex để tìm tất cả các số trong chuỗi
    numbers = re.findall(r'\d+', str(cid_str))
    # Chuyển đổi các số thành chuỗi (vì corpus_ids là danh sách các chuỗi)
    return [str(num) for num in numbers]

In [ ]:
# Load dữ liệu
corpus_texts, corpus_ids = load_corpus(corpus_path)
train_df = pd.read_csv(train_data_path)

# Chọn 50 câu hỏi từ tập train để thử nghiệm
train_sample = train_df.sample(n=50, random_state=42).reset_index(drop=True)

# Áp dụng hàm parse_cid_list vào cột 'cid' của mẫu
train_relevant_cids = train_sample['cid'].apply(parse_cid_list).tolist()
train_queries = train_sample['question'].tolist()

In [ ]:
finetuned_model = SentenceTransformer("hiieu/halong_embedding")

In [ ]:
# Fine-tune lại mô hình Halong Embedding
finetuned_model = finetune_halong_embedding(train_data_path)
print("Đã fine-tune mô hình và lưu checkpoint.")

In [ ]:
# Chuẩn bị dữ liệu cho BM25
print("Đang chuẩn bị dữ liệu cho BM25...")
tokenized_corpus = [text.split() for text in corpus_texts]

# Sử dụng multiprocessing để truy xuất BM25
print("Đang thực hiện truy xuất BM25 cho 50 câu hỏi bằng multiprocessing...")

num_processes = multiprocessing.cpu_count()  # Số lượng tiến trình, bạn có thể điều chỉnh
with multiprocessing.Pool(processes=num_processes, initializer=init_bm25, initargs=(tokenized_corpus, corpus_ids)) as pool:
    retrieved_results = list(tqdm(pool.imap(bm25_retrieve, train_queries), total=len(train_queries)))

In [ ]:
# Tính toán MRR@10 cho 50 câu hỏi với BM25
mrr_bm25 = calculate_mrr(retrieved_results, train_relevant_cids)
print(f"Chỉ số MRR@10 cho 50 câu hỏi với BM25: {mrr_bm25:.4f}")

In [ ]:
# Xếp hạng lại kết quả với mô hình đã fine-tune
reranked_results = []
print("Đang xếp hạng lại kết quả với mô hình đã fine-tune...")
for query, retrieved_docs in tqdm(zip(train_queries, retrieved_results), total=len(train_queries), desc="Reranking"):
    doc_ids = [doc_id for doc_id, _ in retrieved_docs]
    reranked = rerank_with_finetuned_embedding(query, doc_ids, corpus_texts, corpus_ids, finetuned_model)
    reranked_results.append(reranked)

In [ ]:
# Tính toán MRR@10 cho 50 câu hỏi sau khi xếp hạng lại
mrr_finetuned = calculate_mrr(reranked_results, train_relevant_cids)
print(f"Chỉ số MRR@10 cho 50 câu hỏi sau khi fine-tune: {mrr_finetuned:.4f}")

In [ ]:
# # Load dữ liệu
# corpus_texts, corpus_ids = load_corpus(corpus_path)
# train_df = pd.read_csv(train_data_path)
# test_df = pd.read_csv(test_data_path)
# queries = test_df['question'].tolist()
# relevant_ids = test_df['qid'].tolist()  # Lấy danh sách id văn bản liên quan từ test

# Phân tích tập dữ liệu Corpus
# corpus_lengths = pd.Series([len(text.split()) for text in corpus_texts])
# plt.figure(figsize=(10, 6))
# sns.histplot(corpus_lengths[corpus_lengths < 1000], bins=50, kde=True)
# plt.xlabel('Số lượng từ trong văn bản (giới hạn dưới 1000 từ)')
# plt.ylabel('Tần suất')
# plt.title('Phân bổ độ dài của các văn bản trong Corpus (Giới hạn dưới 1000 từ)')
# plt.show()

# # Phân tích tập dữ liệu Train
# train_question_lengths = train_df['question'].apply(lambda x: len(x.split()))
# plt.figure(figsize=(10, 6))
# sns.histplot(train_question_lengths, bins=50, kde=True)
# plt.xlabel('Số lượng từ trong câu hỏi')
# plt.ylabel('Tần suất')
# plt.title('Phân bổ độ dài của câu hỏi trong tập Train')
# plt.show()

# # Phân tích tập dữ liệu Test
# test_question_lengths = test_df['question'].apply(lambda x: len(x.split()))
# plt.figure(figsize=(10, 6))
# sns.histplot(test_question_lengths, bins=50, kde=True)
# plt.xlabel('Số lượng từ trong câu hỏi')
# plt.ylabel('Tần suất')
# plt.title('Phân bổ độ dài của câu hỏi trong tập Test')
# plt.show()

# # So sánh phân bổ độ dài câu hỏi giữa tập Train và Test
# plt.figure(figsize=(12, 6))
# sns.histplot(train_question_lengths, bins=50, kde=True, color='blue', label='Train')
# sns.histplot(test_question_lengths, bins=50, kde=True, color='red', label='Test')
# plt.xlabel('Số lượng từ trong câu hỏi')
# plt.ylabel('Tần suất')
# plt.title('So sánh phân bổ độ dài của câu hỏi giữa tập Train và Test')
# plt.legend()
# plt.show()

# print("Nhận xét:")
# print("- Số lượng câu hỏi khác nhau trong tập Train và Test cho thấy tính đa dạng và tính tổng quát của mô hình.")
# print("- Nếu tập Test có nhiều câu hỏi dài hơn so với tập Train, mô hình có thể gặp khó khăn trong việc dự đoán chính xác vì chưa được huấn luyện đầy đủ với những câu hỏi dài như vậy.")
# print("- Phân bổ độ dài của câu hỏi trong hai tập dữ liệu có thể giúp đánh giá khả năng tổng quát hóa của mô hình.")